# 构建神经网络的基本流程 & 实现过程

## 通用步骤：
- 定义一个拥有可学习参数的神经网络
- 遍历训练数据集
- 处理输入数据使其流经神经网络
- 计算损失值
- 将网络参数的梯度进行反向传播
- 以一定的规则更新网络权重

In [56]:
import torch
import torch.nn as nn # 网络类
import torch.nn.functional as F # 函数包

# 定义一个简单的网络类
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 定义第一层卷积神经网络，输入通道维度 = 1， 输出通道维度 = 6， 卷积核大小3*3
        self.conv1 = nn.Conv2d(1, 6, 3)
        # 定义第二层卷积神经网络，输入通道维度 = 6， 输出通道维度 = 16， 卷积核大小3*3
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 定义三层全连接网络
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 在（2， 2）的池化窗口下执行最大池化操作
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_feature(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
    
    def num_flat_feature(self, x):
        #计算size，除了第0个维度上的batch_size
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s

        return num_features
    
net = Net()

print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [57]:
#所有用pytorch编写的模型都可以通过以下方式获得所有的参数
params = list(net.parameters())

print(len(params))
print(params[0])
print(params[0].size())

10
Parameter containing:
tensor([[[[ 0.1310,  0.0218, -0.2810],
          [-0.1165,  0.3243, -0.3321],
          [-0.3293, -0.2957, -0.0076]]],


        [[[ 0.2938,  0.3283, -0.1788],
          [ 0.2150, -0.0640,  0.1818],
          [ 0.1402,  0.0028,  0.2634]]],


        [[[ 0.1786,  0.1135,  0.1287],
          [ 0.1005,  0.3207, -0.0319],
          [ 0.2394, -0.0185,  0.2673]]],


        [[[ 0.1848, -0.1282,  0.2584],
          [-0.1204,  0.2060, -0.3110],
          [-0.1760,  0.1501,  0.3007]]],


        [[[ 0.0768, -0.0212, -0.0919],
          [-0.1376,  0.1893, -0.0542],
          [ 0.0176, -0.0036, -0.2132]]],


        [[[-0.0954,  0.0960, -0.2430],
          [-0.1170,  0.2015,  0.1173],
          [ 0.1192,  0.1346,  0.1111]]]], requires_grad=True)
torch.Size([6, 1, 3, 3])


In [58]:
# 假设图像输入尺寸为32 * 32
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.1123,  0.0256,  0.1281,  0.0875, -0.0965,  0.0398,  0.0189, -0.0705,
         -0.0600,  0.1112]], grad_fn=<AddmmBackward0>)


In [59]:
# 有了输出的张量之后，就可以执行梯度归零和反向传播的操作了
net.zero_grad()
out.backward(torch.randn(1, 10))

# 损失函数

## 输入是一个pair：（output, target），然后计算出一个数值来评估output和target之间的差距大小

In [60]:
output = net(input)
target = torch.randn(10)

# 改变target的形状为二维张量，为了和output匹配
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6072, grad_fn=<MseLossBackward0>)


## 以上神经网络完整的传播链条

input  
      -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d  
      -> view -> linear -> relu -> linear -> relu -> linear  
      -> MSELoss  
      -> loss  

#### 当我们调用loss.backward()时，整张计算图将对loss进行自动求导，所有属性requires_grad = True的Tensor都将参与梯度求导的运算，并将梯度累加到Tensors中的.grad属性中

In [61]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])


In [62]:
# 每个批次必须都要先清零梯度
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0026, -0.0123,  0.0346,  0.0280, -0.0240, -0.0034])


## 更新网络参数

更新参数最简单的算法就是SGD  
具体的公式： weight = weight - learning_rate * gradient

In [64]:
# learning_rate = 0.01

# # 传统python代码实现SGD
# for f in net.parameters():
#     f.data.sub_(f.grad.data * learning_rate)



# pytorch官方推荐的标准代码

# 首先导入优化器包，其中包含了若干的常用优化算法，例如SGD， Adam等
import torch.optim as optim

# 通过optim创建优化器对象
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# 将优化器执行梯度清零操作
# ***写死（清零）
optimizer.zero_grad()

output = net(input)
loss = criterion(output, target)

# 对损失值执行反向传播操作(只是把梯度都算出来了而已)
# ***写死（计算）
loss.backward()

# 参数的更新通过一行标准代码来执行（才是在执行更新）
# ***写死（更新）
optimizer.step()